In [1]:
import pandas as pd

In [2]:
file1 = "UkraineCombinedTweetsDeduped_FEB27.csv.gzip"
file2 = "UkraineCombinedTweetsDeduped_FEB28_part2.csv.gzip"
file3 = "UkraineCombinedTweetsDeduped_MAR01.csv.gzip"
file4 = "UkraineCombinedTweetsDeduped_MAR02.csv.gzip"
file5 = "UkraineCombinedTweetsDeduped_MAR03.csv.gzip"
file6 = "UkraineCombinedTweetsDeduped_MAR04.csv.gzip"
file7 = "UkraineCombinedTweetsDeduped_MAR05.csv.gzip"


In [3]:
temp_table1 = pd.read_csv(file1, compression='gzip')
temp_table2 = pd.read_csv(file2, compression='gzip')
temp_table3 = pd.read_csv(file3, compression='gzip')
temp_table4 = pd.read_csv(file4, compression='gzip')
temp_table5 = pd.read_csv(file5, compression='gzip')
temp_table6 = pd.read_csv(file6, compression='gzip')
temp_table7 = pd.read_csv(file7, compression='gzip')

C:\Users\aglas\AppData\Local\Temp\ipykernel_20020\2672232526.py:6: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  temp_table6 = pd.read_csv(file6, compression='gzip')
C:\Users\aglas\AppData\Local\Temp\ipykernel_20020\2672232526.py:7: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  temp_table7 = pd.read_csv(file7, compression='gzip')


In [4]:
df = temp_table1.append([temp_table2,temp_table3,temp_table4,temp_table5,temp_table6,temp_table7])

C:\Users\aglas\AppData\Local\Temp\ipykernel_20020\2091050160.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = temp_table1.append([temp_table2,temp_table3,temp_table4,temp_table5,temp_table6,temp_table7])


In [5]:
df = df[df.language == 'en']
df = df[['location','text','hashtags','tweetcreatedts']]
df = df.loc[df.hashtags.str.contains('Putin') | df.hashtags.str.contains('Zelensky')]

In [11]:
print(df['text'])

27        #Putin will not stop at #Ukraine.\n\nFuck Puti...
36        Kharkov right now! 🇺🇦\n\n#Ukraine #UkraineRuss...
74        Is this Putin’s Napoleon moment? Let’s pray so...
101       Live: #Ukraine vows to fight on as #Kyiv readi...
120       I'm perplexed that people keep throwing out th...
                                ...                        
546738    @Diop_IFC @techreview #Ukraine needs weapons a...
546754    @ChrisVanHollen #Ukraine needs weapons and hum...
546768    @RoyalFamily #Ukraine needs weapons and humani...
546774    #Putin will not stop with #Ukraine he does not...
546775    @sven_giegold @LiberecoPHR @spirit_andsoul #Uk...
Name: text, Length: 350195, dtype: object


In [45]:
import warnings
warnings.filterwarnings("ignore")

import re

import simplemma

from spacy.lang.en import English

langdata = dict()
langdata['en'] = simplemma.load_data('en')   

nlp = English()
lang = langdata.get('en')

def no_html(raw_text):
    
    no_html = raw_text.lower().strip()

    # remove urls
    no_html = re.compile('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+').sub(r'', no_html)
    no_html = re.sub('www.*.(com|hr)', '', no_html)
    
    # remove emoji
    no_html = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE).sub(r'', no_html)

    # remove quotations, re: fw: and aw:
    no_html = re.sub('"', ' ', no_html)
    no_html = re.sub("'", ' ', no_html)
   
    no_html = re.sub('re:|fw:|aw:', ' ', no_html)



    no_html = re.sub("from:.*", '', no_html)


    # remove special symbols, incorrect words and numbers
    no_html = re.compile('[/(){}\[\]<>\|@,;_–.?!-*]').sub(' ', no_html)
    no_html = re.compile('[\d+]').sub(' ', no_html)
    no_html = re.sub(":", ' ', no_html)
    no_html = re.sub("-", ' ', no_html)
    no_html = re.sub(r'\s+',' ', no_html)
    no_html = no_html.strip()
    
    
    return no_html



def process_text_with_original(no_html):
    if no_html is None:
        return None

    # Second version (returns a string)
    cleaned_doc = [simplemma.lemmatize(token.text, lang).lower() for token in nlp(no_html) if len(token.text) > 1 and not token.is_stop and not token.is_punct and token.is_alpha]
    cleaned_doc = ' '.join(cleaned_doc)

    return cleaned_doc

In [46]:
df['text'] = df['text'].apply(str)

In [47]:
df['text'] = df['text'].map(no_html)

In [59]:
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
stop = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aglas\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [60]:
df['text'] = df['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

In [62]:
df[['text']]

,text
27,putin stop ukraine fuck putin fckputin
36,kharkov right ukraine ukrainerussiawar ukrinia...
74,putin’s napoleon moment let’s pray putin ukrai...
101,live ukraine vows fight kyiv readies russian m...
120,perplexed people keep throwing zelenskyy used ...
...,...
546738,diop ifc techreview ukraine needs weapons huma...
546754,chrisvanhollen ukraine needs weapons humanitar...
546768,royalfamily ukraine needs weapons humanitarian...
546774,putin stop ukraine care wwiii nato
